In [155]:
import datetime
import subprocess
# from fabric import Connection
import pandas as pd
# import paramiko
import math
from scipy import stats
import numpy as np
import os
import warnings
import time
import random

# Ignore all warnings
warnings.filterwarnings("ignore")

In [156]:
def get_power_data(start_time, end_time):
    power_data = []
    time_period = (end_time - start_time).seconds

    for i in range(time_period):
        power = 80 + (random.random() * 180)
        timestamp = start_time + datetime.timedelta(seconds=i)
        power_data.append((timestamp, power))
        time.sleep(1)

    df_power = pd.DataFrame(power_data, columns=['time_stamp', 'power'])
    return df_power

# # Example usage
# start_time = datetime.datetime(2023, 5, 25, 10, 0, 0)
# end_time = datetime.datetime(2023, 5, 25, 10, 0, 10)
# power_data = get_power_data(start_time, end_time)
# print(power_data)

In [157]:
def copy_file(power_log):
    # power_log = pd.read_csv("../metrics/energy_log/power_log_2023-05-15.csv", names=['time_stamp', 'power'])
    power_log = power_log
    power_log['time_stamp'] = pd.to_datetime(power_log['time_stamp'])
    # print(power_log.head())
    return power_log

In [158]:
def get_nearest_power(start_time, end_time):
    # Copy the file from the server to another server
    power_log = copy_file(get_power_data(start_time, end_time))
    power_log = power_log.set_index('time_stamp')  # Set date to index

    index_start = power_log.index.get_indexer([start_time], method='nearest')[0]  # new version
    index_end = power_log.index.get_indexer([end_time], method='nearest')[0]

    power_log = power_log.reset_index()
    power_select = power_log.loc[index_start:index_end, ['time_stamp', 'power']]  # one到two行，ac列
    time_list = power_select['time_stamp'].tolist()
    power_list = power_select['power'].tolist()

    return power_list, time_list

In [159]:
def meets_condition1_poweridle(power_list):
    # Average idle power
    power_idle = 80
    # Threshold value
    threshold = 1.05

    average_power = sum(power_list) / len(power_list)
    return average_power > power_idle * threshold

In [160]:
def meets_condition2_tdistribution(power_list):
    alpha = 0.05
    n = len(power_list)  # Sample size
    sample_std = np.std(power_list, ddof=1)
    sample_mean = np.mean(power_list)

    t_alpha_half = stats.t.ppf(1 - alpha / 2, df=n - 1)

    left_side = (t_alpha_half * sample_std) / (2 * alpha * sample_mean)
    right_side = n * math.sqrt(2 * n)
    # If the criteria are met, return True; otherwise, return False
    return left_side < right_side, left_side, right_side

In [ ]:
# method test2
def measure_command_runtime_distribution(cmd):
    count_encode = 0
    encoding_processes = {}
    consecutive_failures = 0
    total_power_list = []

    while True:
        start_time_encode = datetime.datetime.now()
        subprocess.run(cmd, encoding="utf-8", shell=True)
        end_time_encode = datetime.datetime.now()

        current_power_list, time_list = get_nearest_power(start_time_encode, end_time_encode) #current power

        if meets_condition1_poweridle(current_power_list):
            total_power_list.append(current_power_list)
            count_encode += 1
            encoding_processes[count_encode] = (current_power_list, time_list)
            consecutive_failures = 0
            # put a logging (keep the measurement of formula inequality sides)
            measure, left_side, right_side = meets_condition2_tdistribution(total_power_list)
            if measure: #
                break
            # else: # count_encode >100 we break the loop #take the logging notes
            #     # consecutive_failures += 1 # measurement count_encode = 100
        else:
            consecutive_failures += 1

        if consecutive_failures >= 5:
            break

    return encoding_processes

In [162]:
codec = 'SVT-AV1'

if codec == 'SVT-AV1':
    codec_name = 'libsvtav1'  # 'libaom-av1' #'libsvtav1'
    qp_level = [12]
    # qp_level = [12, 18, 24, 30, 36, 42, 48, 54, 60]

df_metrics = pd.DataFrame(columns=['vid', 'QP', 'start_time_encode', 'end_time_encode', 'count_encode'])

vid_list = []
qp_list = []
count_encode_list = []
start_time_encode_list = []
end_time_encode_list = []

for qp in qp_level:
    vid_list.append('TelevisionClip_1080P-68c6')
    qp_list.append(qp)

    encode_command = f'ffmpeg -s 1920x1080 -r 25.0 -pix_fmt yuv420p -y -i ../videocodec/input/TelevisionClip_1080P-68c6.yuv -c:v libsvtav1 -crf {str(qp)} ../videocodec/encoded/SVT-AV1/TelevisionClip_1080P-68c6_encoded_fps25.0_crf_{str(qp)}.mp4 1>../videocodec/encoded/SVT-AV1/encoded_test_log_crf_{str(qp)}.txt 2>&1'
    encoding_processes = measure_command_runtime_distribution(encode_command)
    print(encoding_processes)

    # start_time, end_time, count_encode = measure_command_runtime_distribution(encode_command)
    #
    # print(f'start_time: {start_time}')
    # print(f'end_time: {end_time}')
    # print(f'count_encode: {count_encode}')
    #
    # count_encode_list.append(count_encode)
    # start_time_encode_list.append(start_time)
    # end_time_encode_list.append(end_time)

    os.remove(f'../videocodec/encoded/SVT-AV1/TelevisionClip_1080P-68c6_encoded_fps25.0_crf_{str(qp)}.mp4')

# df_metrics['vid'] = vid_list
# df_metrics['QP'] = qp_list
# df_metrics['count_encode'] = count_encode_list
# df_metrics['start_time_encode'] = start_time_encode_list
# df_metrics['end_time_encode'] = end_time_encode_list
#
# print(df_metrics)
# metrics_name = f'../metrics/energy_log/YOUTUBE_UGC_1080P_{codec}_distribution_test.csv'
# df_metrics.to_csv(metrics_name, index=None)

[248.2475533322396, 137.0060197624896, 119.86062657132672, 204.13633705877027, 248.8442728471061, 242.93117060128577, 240.99503059591262, 140.41039766845296, 140.6495552934407, 186.96227072940803, 164.42040530304388, 234.5245700854154, 195.45546717903875, 116.93277554165189, 232.8982972961488]
--------------------------------------------------------------------
[Timestamp('2023-06-01 04:58:10.829528'), Timestamp('2023-06-01 04:58:11.829528'), Timestamp('2023-06-01 04:58:12.829528'), Timestamp('2023-06-01 04:58:13.829528'), Timestamp('2023-06-01 04:58:14.829528'), Timestamp('2023-06-01 04:58:15.829528'), Timestamp('2023-06-01 04:58:16.829528'), Timestamp('2023-06-01 04:58:17.829528'), Timestamp('2023-06-01 04:58:18.829528'), Timestamp('2023-06-01 04:58:19.829528'), Timestamp('2023-06-01 04:58:20.829528'), Timestamp('2023-06-01 04:58:21.829528'), Timestamp('2023-06-01 04:58:22.829528'), Timestamp('2023-06-01 04:58:23.829528'), Timestamp('2023-06-01 04:58:24.829528')]
[131.94715595089772,